此代码实现了迭代的三种方法。Jacobi Gauss-Seidel 以及 SOR 的迭代。

案例分析使用的是

A = [[5, 2, 1], [-1, 4, 2], [2, -3, 10]]

b = [-12, 20, 3]

x = [100, 100, 100]

SOR的omega松弛因子可以进行修改，目前设置值为0.5，下一次更新会被前值正向影响。

因为x是三维向量，所以同时对三个迭代方法进行了三维可视化，

可以看到，Gauss-Seidel 相较于 Jacobi 收敛更快，对于不同的omega，SOR的收敛速度不同，且还和初值设置有关。


In [3]:

import numpy as np

A = [[5, 2, 1], [-1, 4, 2], [2, -3, 10]]
b = [-12, 20, 3]
# x = np.zeros_like(b)
x = [100, 100, 100]


def Jacobi(A, b, x):
    data = []
    D = np.diag(np.diag(A))
    L = -np.tril(A, k=-1)
    U = -np.triu(A, k=1)

    B = np.linalg.inv(D) @ (L + U)
    g = np.linalg.inv(D) @ b

    data.append({'x1': float(x[0]), 'x2': float(x[1]), 'x3': float(x[2])})
    # print("初始猜测解:", x)
    for i in range(8):
        x = B @ x + g
        data.append({'x1': float(x[0]), 'x2': float(x[1]), 'x3': float(x[2])})
        # print(f"第{i+1}次迭代结果:", x)
    return data


def Gauss_Seidel(A, b, x):
    data = []
    D = np.diag(np.diag(A))
    L = -np.tril(A, k=-1)
    U = -np.triu(A, k=1)

    B = np.linalg.inv(D - L) @ U
    g = np.linalg.inv(D - L) @ b

    data.append({'x1': float(x[0]), 'x2': float(x[1]), 'x3': float(x[2])})
    # print("初始猜测解:", x)
    for i in range(8):
        x = B @ x + g
        data.append({'x1': float(x[0]), 'x2': float(x[1]), 'x3': float(x[2])})
        # print(f"第{i+1}次迭代结果:", x)
    return data


def SOR(A, b, x, omega):
    data = []
    D = np.diag(np.diag(A))
    L = -np.tril(A, k=-1)
    U = -np.triu(A, k=1)

    B = np.linalg.inv(D - L) @ U
    g = np.linalg.inv(D - L) @ b

    data.append({'x1': float(x[0]), 'x2': float(x[1]), 'x3': float(x[2])})
    # print("初始猜测解:", x)
    for i in range(8):
        xt = B @ x + g
        x = np.diag([omega] * len(x)) @ xt + np.diag([1 - omega] * len(x)) @ x
        data.append({'x1': float(x[0]), 'x2': float(x[1]), 'x3': float(x[2])})
        # print(f"第{i+1}次迭代结果:", x)
    return data


omega = 0.5
j_data = Jacobi(A, b, x)
gs_data = Gauss_Seidel(A, b, x)
sor_data = SOR(A, b, x, omega)
# print(data)

import plotly.graph_objects as go

# 从字典数据提取坐标
jacobi_x1 = [point['x1'] for point in j_data]
jacobi_x2 = [point['x2'] for point in j_data]
jacobi_x3 = [point['x3'] for point in j_data]
gauss_seidel_x1 = [point['x1'] for point in gs_data]
gauss_seidel_x2 = [point['x2'] for point in gs_data]
gauss_seidel_x3 = [point['x3'] for point in gs_data]
sor_x1 = [point['x1'] for point in sor_data]
sor_x2 = [point['x2'] for point in sor_data]
sor_x3 = [point['x3'] for point in sor_data]

# 创建3D散点图 - 修复color参数
fig = go.Figure(data=[
    go.Scatter3d(
        name='Jacobi',
        mode='lines+markers+text',
        x=jacobi_x1, y=jacobi_x2, z=jacobi_x3,
        text=['J_' + str(i) for i in range(len(j_data))],
        marker=dict(
            size=2,
            color='red',
        ),
        line=dict(color='red', width=2),
    ),
    go.Scatter3d(
        name='Gauss-Seidel',
        mode='lines+markers+text',
        x=gauss_seidel_x1, y=gauss_seidel_x2, z=gauss_seidel_x3,
        text=['GS_' + str(i) for i in range(len(j_data))],
        marker=dict(
            size=2,
            color='green',
        ),
        line=dict(color='green', width=2),
    ),
    go.Scatter3d(
        name=f'SOR-omega-{omega}',
        mode='lines+markers+text',
        x=sor_x1, y=sor_x2, z=sor_x3,
        text=['SOR_' + str(i) for i in range(len(j_data))],
        marker=dict(
            size=2,
            color='blue',
        ),
        line=dict(color='blue', width=2),
    ),
])

# 更新布局
fig.update_layout(
    title='【Jacobi & Gauss—Seidel & SOR】迭代法收敛路径',
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='x3',
    ),
)

# 显示图形
fig.show()

# 保存图形为HTML文件以便查看
fig.write_html("iteration.html")
# print("三维可视化完成！图形已保存为 'iteration.html'")

